In [1]:
!pip install Openai
!pip install faiss-cpu
!pip install PyPDF2

In [79]:
import re
import openai as OpenAI

In [80]:
import re
from openai import OpenAI

raw = open("openaikey.txt", "r").read()
m = re.search(r"(sk-[A-Za-z0-9\-_]+)", raw)
if not m:
    raise RuntimeError("No API key found in openaikey.txt — put the raw key or adjust parsing.")
api_key = m.group(1)
#print("Using key:", repr(api_key))   # sanity check
client = OpenAI(api_key=api_key)

resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role":"user","content":"ping"}]
)
print(resp.choices[0].message.content)

Pong! How can I assist you today?


In [81]:

import PyPDF2              
import faiss                 
import numpy as np           
from openai import OpenAI

In [82]:
def read(path):
    reader=PyPDF2.PdfReader(path)
    text=""
    for page in reader.pages:
        pagetext=page.extract_text()
        if pagetext:
            text=text+pagetext
    return text
    

In [83]:
read("kalam.pdf")

'Dr. APJ Abdul Kala m \nwww.abdulkalam.com  \nPage 1 of 5 Profile of Dr. APJ Abdul Kalam  \n \n Born on 15th October 1931 at \nRameswaram in Tamil Nadu, Dr.  Avul \nPakir Jainulabdeen Abdul Kalam, \ngraduated in Science from St. \nJoseph’s College, Trichy in 1954 and \nspecialized in Aeronautical \nEngineering from Madras Institu te of \nTechnology (MIT) in 1957. Dr. Kalam \nis a pioneer in fibre glass technology and led a young team to \ninitiate this effort in ISRO from design, development leading to \nproduction of composite s rocket motor cases . Dr. Kalam made \nsignificant contribution a s Project Director to develop India’s \nfirst indigenous Satellite Launch Vehicle (SLV -3) which \nsuccessfully injected the Rohini satellite in the near earth orbit \nin July 1980 and made India an exclusive member of Space \nClub.  \nHe was responsible for the evolu tion of ISRO\'s launch \nvehicle programme, particularly the PSLV configuration. After \nworking for two decades in ISRO and maste

In [84]:
pdf=read("kalam.pdf")
len(pdf)

7103

In [85]:
def seprating(paragraph,chunk_size=480):
    chunkslist=[]
    start=0
    while start<len(pdf):
        end=start+chunk_size
        chunk=paragraph[start:end]
        chunkslist.append(chunk)
        start=end
    return chunkslist

        


In [86]:
a=seprating(pdf)
len(a)


15

In [87]:
a[0]

'Dr. APJ Abdul Kala m \nwww.abdulkalam.com  \nPage 1 of 5 Profile of Dr. APJ Abdul Kalam  \n \n Born on 15th October 1931 at \nRameswaram in Tamil Nadu, Dr.  Avul \nPakir Jainulabdeen Abdul Kalam, \ngraduated in Science from St. \nJoseph’s College, Trichy in 1954 and \nspecialized in Aeronautical \nEngineering from Madras Institu te of \nTechnology (MIT) in 1957. Dr. Kalam \nis a pioneer in fibre glass technology and led a young team to \ninitiate this effort in ISRO from design, developme'

In [88]:
a[1]

"nt leading to \nproduction of composite s rocket motor cases . Dr. Kalam made \nsignificant contribution a s Project Director to develop India’s \nfirst indigenous Satellite Launch Vehicle (SLV -3) which \nsuccessfully injected the Rohini satellite in the near earth orbit \nin July 1980 and made India an exclusive member of Space \nClub.  \nHe was responsible for the evolu tion of ISRO's launch \nvehicle programme, particularly the PSLV configuration. After \nworking for two decades i"

In [89]:
a[2]

'n ISRO and mastering launch vehicle \ntechnologies, Dr. Kalam took up the responsibility of \ndeveloping Indigenous Guided Missiles at Defence Research  \nand Development Organisation as the Chief Executive of \nIntegrated Guided Missile Development Programme (IGMDP). \nHe was responsible for the development and operationalisation \nDr. APJ Abdul Kala m \nwww.abdulkalam.com  \nPage 2 of 5 of AGNI and PRITHVI Missiles and for building indigenous \ncapability in critical t echnologies t'

In [90]:
def getembeddings(chunks):
    embeddings=[]
    for chunk in chunks:
         response = client.embeddings.create(input=chunk,model="text-embedding-3-small")
         vectors = response.data[0].embedding
         embeddings.append(vectors)
        
    return np.array(embeddings).astype("float32")
    

In [91]:
def convert_embedding(embeddings):
    dim=len(embeddings[0])
    index=faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index
    

In [92]:
def questions(question,chunks,index):
    que_embed=client.embeddings.create(input=question,
                                      model="text-embedding-3-small"
                                     ).data[0].embedding
    ques=np.array([que_embed]).astype("float32")
    distances,indices=index.search(ques,k=2)
    print(len(indices[0]))
    best_chunks = [chunks[i] for i in indices[0]]
    print(len(best_chunks))
    return best_chunks
    

In [93]:
def ask_question(chunks , question):
    context = "\n\n".join(chunks)
    prompt = f"Answer the question based on this text \n{context} \n\n {question}"

    response = client.chat.completions.create(model = "gpt-4o-mini",messages = [{"role":"user","content":prompt}])
    return response.choices[0].message.content



In [95]:
 # 1. Read text from a sample PDF
text = read("kalam.pdf")
print("✅ PDF Loaded.")

    # 2. Split text into smaller chunks
chunks =seprating(text)
print("✅ Text divided into chunks.")

    # 3. Create embeddings for all chunks
embeddings = getembeddings(chunks)
print("✅ Embeddings created.")

    # 4. Store embeddings in FAISS
index = convert_embedding(embeddings)
print("✅ FAISS index created.")

    # 5. Ask user for a question
question = input("Ask your Question from PDF : ")

    # 6. Find similar text chunks
best_chunks = questions(question,chunks,index)
print("✅ Found related text parts.")
print(best_chunks)
print("******************************")

    # 7. Get final answer from OpenAI
result= ask_question(best_chunks , question)
print("\n🧠 Answer from OpenAI:")
print(result)

✅ PDF Loaded.
✅ Text divided into chunks.
✅ Embeddings created.
✅ FAISS index created.


Ask your Question from PDF :  how are you 


2
2
✅ Found related text parts.
['n Wings Award ” in September 2009.   \n \nDr. Kalam became the 11th President of India on 25th \nJuly 2002. After five eventful years he demitted offi ce on 25th \nJuly 2007.   His focus is on transforming India into a developed \nnation by 2020.  His accent is on constructive networking and \nexcellent human resources for a n economically developed,  \nprosperous and peaceful society.  ', 'ology in \nOctober 2007 .  He received the Woodrow Wilson Award  in \n2008.  The Royal Academy of  Engineering, London conferred \non him the International Medal 2008  in June 2009 at \nLondon.  The Hoover Board of Awards presented him the \nHoover Medal 2008  at New York in April 2009.   The \nAerospace Historical Society  in Collaboration with the \nGraduate  Aeros pace Laboratories (GALCIT) at the California \nInstitute of Technology  awarded him the “2009 International \nVon Karma']
******************************

🧠 Answer from OpenAI:
I'm just a computer progra